<a href="https://colab.research.google.com/github/Serena428/Rader_performance_prediction/blob/main/Test3_model_sy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import numpy as np
import warnings

from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet
from sklearn.kernel_ridge import KernelRidge

from sklearn.ensemble import BaggingRegressor

from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import StackingRegressor

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error

# Global settings

import plotly.express as px
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns


warnings.filterwarnings("ignore") # To ignore warnings
n_jobs = -1 # This parameter conrols the parallel processing. -1 means using all processors.
random_state = 42 # This parameter controls the randomness of the data. Using some int value to get same results everytime this code is run.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df=pd.read_csv('/content/drive/MyDrive/수연/study/radar/train.csv')
#train_df.head()
len(df)

39607

In [ ]:
test_df=pd.read_csv('/content/drive/MyDrive/수연/study/radar/test.csv')
#train_df.head()
len(test_df)

39608

In [ ]:
df.head()

,ID,X_01,X_02,X_03,X_04,X_05,X_06,X_07,X_08,X_09,...,Y_05,Y_06,Y_07,Y_08,Y_09,Y_10,Y_11,Y_12,Y_13,Y_14
0,TRAIN_00001,70.544,103.320,67.47,1,101.892,74.983,29.45,62.38,245.71,...,29.632,16.083,4.276,-25.381,-25.529,-22.769,23.792,-25.470,-25.409,-25.304
1,TRAIN_00002,69.524,103.321,65.17,1,101.944,72.943,28.73,61.23,233.61,...,33.179,16.736,3.229,-26.619,-26.523,-22.574,24.691,-26.253,-26.497,-26.438
2,TRAIN_00003,72.583,103.320,64.07,1,103.153,72.943,28.81,105.77,272.20,...,31.801,17.080,2.839,-26.238,-26.216,-22.169,24.649,-26.285,-26.215,-26.370
3,TRAIN_00004,71.563,103.320,67.57,1,101.971,77.022,28.92,115.21,255.36,...,34.503,17.143,3.144,-25.426,-25.079,-21.765,24.913,-25.254,-25.021,-25.345
4,TRAIN_00005,69.524,103.320,63.57,1,101.981,70.904,29.68,103.38,241.46,...,32.602,17.569,3.138,-25.376,-25.242,-21.072,25.299,-25.072,-25.195,-24.974


In [ ]:
df.columns

Index(['ID', 'X_01', 'X_02', 'X_03', 'X_04', 'X_05', 'X_06', 'X_07', 'X_08',
       'X_09', 'X_10', 'X_11', 'X_12', 'X_13', 'X_14', 'X_15', 'X_16', 'X_17',
       'X_18', 'X_19', 'X_20', 'X_21', 'X_22', 'X_23', 'X_24', 'X_25', 'X_26',
       'X_27', 'X_28', 'X_29', 'X_30', 'X_31', 'X_32', 'X_33', 'X_34', 'X_35',
       'X_36', 'X_37', 'X_38', 'X_39', 'X_40', 'X_41', 'X_42', 'X_43', 'X_44',
       'X_45', 'X_46', 'X_47', 'X_48', 'X_49', 'X_50', 'X_51', 'X_52', 'X_53',
       'X_54', 'X_55', 'X_56', 'Y_01', 'Y_02', 'Y_03', 'Y_04', 'Y_05', 'Y_06',
       'Y_07', 'Y_08', 'Y_09', 'Y_10', 'Y_11', 'Y_12', 'Y_13', 'Y_14'],
      dtype='object')

In [ ]:
X = df[['X_01', 'X_02', 'X_03', 'X_04', 'X_05', 'X_06', 'X_07', 'X_08',
       'X_09', 'X_10', 'X_11', 'X_12', 'X_13', 'X_14', 'X_15', 'X_16', 'X_17',
       'X_18', 'X_19', 'X_20', 'X_21', 'X_22', 'X_23', 'X_24', 'X_25', 'X_26',
       'X_27', 'X_28', 'X_29', 'X_30', 'X_31', 'X_32', 'X_33', 'X_34', 'X_35',
       'X_36', 'X_37', 'X_38', 'X_39', 'X_40', 'X_41', 'X_42', 'X_43', 'X_44',
       'X_45', 'X_46', 'X_47', 'X_48', 'X_49', 'X_50', 'X_51', 'X_52', 'X_53',
       'X_54', 'X_55', 'X_56']]

In [ ]:
y = df[['Y_01', 'Y_02', 'Y_03', 'Y_04', 'Y_05', 'Y_06',
       'Y_07', 'Y_08', 'Y_09', 'Y_10', 'Y_11', 'Y_12', 'Y_13', 'Y_14']]

#Train and Test Data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size = 0.33, random_state=random_state)

print(f'Training set--> X_train shape= {X_train.shape}, y_train shape= {y_train.shape}')
print(f'Holdout set--> X_test shape= {X_test.shape}, y_test shape= {y_test.shape}')

Training set--> X_train shape= (26536, 56), y_train shape= (26536, 14)
Holdout set--> X_test shape= (13071, 56), y_test shape= (13071, 14)


# Regression

In [ ]:
# def lg_nrmse(gt, preds):
#     # 각 Y Feature별 NRMSE 총합
#     # Y_01 ~ Y_08 까지 20% 가중치 부여
#     all_nrmse = []
#     for idx in range(1,15): # ignore 'ID'
#         rmse = metrics.mean_squared_error(gt[:,idx], preds[:,idx], squared=False)
#         nrmse = rmse/np.mean(np.abs(gt[:,idx]))
#         all_nrmse.append(nrmse)
#     score = 1.2 * np.sum(all_nrmse[:8]) + 1.0 * np.sum(all_nrmse[8:14])
#     return score

In [ ]:
models_scores = [] # To store model scores

def rmse(model):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    return mean_squared_error(y_test, y_pred, squared= False) # squared= False > returns Root Mean Square Error 

##Linear Regression

In [ ]:
lr = LinearRegression()
lr.fit(X_train,y_train)

LinearRegression()

In [ ]:
test_df.head()
test_df.set_index('ID')

,X_01,X_02,X_03,X_04,X_05,X_06,X_07,X_08,X_09,X_10,...,X_47,X_48,X_49,X_50,X_51,X_52,X_53,X_54,X_55,X_56
ID,,,,,,,,,,,,,,,,,,,,,
TEST_00001,68.504,103.321,76.67,1,101.867,73.963,30.51,63.57,239.80,0.0,...,1,1,17227.63,138.130429,129.460682,141.506570,133.427229,129.711498,133.138096,121.859684
TEST_00002,67.485,103.320,69.37,1,101.992,67.845,28.03,116.99,189.23,0.0,...,1,1,17134.53,136.148839,128.266277,145.911745,131.196417,132.411480,133.629025,124.178623
TEST_00003,69.524,103.320,68.97,1,101.884,77.022,29.65,205.68,214.93,0.0,...,1,1,14860.83,120.447446,119.988804,132.099908,120.450155,130.051708,128.252972,114.475628
TEST_00004,69.524,103.320,65.87,1,101.866,73.963,28.15,103.38,180.80,0.0,...,1,1,15252.53,133.994695,125.069180,147.507669,123.142653,125.963665,139.666592,126.589253
TEST_00005,73.603,103.321,66.67,1,101.891,74.983,29.92,71.20,231.93,0.0,...,1,1,10752.23,137.918202,135.116192,138.600473,127.173033,137.252712,134.411335,124.020016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TEST_39604,68.504,103.320,63.97,1,103.157,68.864,29.49,116.35,284.16,0.0,...,1,1,62123.53,127.741246,126.494312,139.119905,125.271109,128.284572,140.176945,128.292843
TEST_39605,68.504,103.320,61.37,1,103.137,68.864,32.29,116.28,272.41,0.0,...,1,1,61844.13,127.767377,124.062809,138.238664,119.879393,127.322529,137.312047,131.570614
TEST_39606,69.524,103.320,63.67,1,103.149,69.884,30.00,113.05,295.54,0.0,...,1,1,60277.53,128.593640,124.774037,138.659624,123.999571,126.075542,135.656132,127.671108


In [ ]:
test_df=test_df[['X_01', 'X_02', 'X_03', 'X_04', 'X_05', 'X_06', 'X_07', 'X_08',
       'X_09', 'X_10', 'X_11', 'X_12', 'X_13', 'X_14', 'X_15', 'X_16', 'X_17',
       'X_18', 'X_19', 'X_20', 'X_21', 'X_22', 'X_23', 'X_24', 'X_25', 'X_26',
       'X_27', 'X_28', 'X_29', 'X_30', 'X_31', 'X_32', 'X_33', 'X_34', 'X_35',
       'X_36', 'X_37', 'X_38', 'X_39', 'X_40', 'X_41', 'X_42', 'X_43', 'X_44',
       'X_45', 'X_46', 'X_47', 'X_48', 'X_49', 'X_50', 'X_51', 'X_52', 'X_53',
       'X_54', 'X_55', 'X_56']]

In [ ]:
pred=lr.predict(test_df)

In [ ]:
len(test_df)

39608

In [ ]:
len(pred)

39608

In [ ]:
linear_regression = make_pipeline(LinearRegression())

#LinearRegression.fit(X_train, y_train)
y_pred = LinearRegression.predict(X_test)
score = rmse(linear_regression)
models_scores.append(['Linear Regression', score])
print(f'LinearRegression Score={score}')

TypeError: ignored

In [ ]:
X_test

,X_01,X_02,X_03,X_04,X_05,X_06,X_07,X_08,X_09,X_10,...,X_47,X_48,X_49,X_50,X_51,X_52,X_53,X_54,X_55,X_56
17963,66.465,103.32,62.57,1,102.080,67.845,28.91,101.23,260.04,0.0,...,1,1,12604.63,128.065556,142.927876,143.934663,127.675734,128.842655,134.760060,126.813557
14237,70.544,103.32,78.17,1,101.929,71.923,28.86,111.90,214.75,0.0,...,1,1,15321.23,127.643056,133.561871,140.044099,127.529585,126.330578,133.354809,133.543911
2992,68.504,103.32,68.17,1,103.136,68.864,28.14,108.46,364.07,0.0,...,1,1,12411.43,136.358098,129.140545,132.645387,126.858297,124.748920,128.367512,121.985188
18587,74.623,103.32,81.37,1,103.157,74.983,30.13,376.16,188.52,0.0,...,1,1,14008.93,131.659291,125.397381,148.744196,131.340460,135.328876,144.221821,130.223950
19112,65.445,103.32,64.57,1,101.986,67.845,27.11,103.94,223.55,0.0,...,1,1,10530.03,132.419370,129.721226,143.947633,132.284460,126.447689,146.282254,133.663632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25593,67.485,103.32,67.27,1,101.898,70.904,28.00,113.20,188.34,0.0,...,1,1,11847.73,143.714902,134.504200,137.821771,123.207068,121.834835,137.007130,131.626667
12105,68.504,103.32,68.47,1,103.154,68.864,27.71,125.72,245.00,0.0,...,1,1,12285.53,137.372418,140.266583,145.743127,132.552496,133.956959,146.378743,130.425694
32028,73.603,103.32,64.87,1,103.159,73.963,26.83,103.04,267.78,0.0,...,1,1,13966.63,142.462094,137.324070,155.025171,137.003256,135.016773,155.004887,135.983715
26645,68.504,103.32,63.67,1,103.153,68.864,29.26,102.02,268.41,0.0,...,1,1,9790.73,126.876691,120.843311,131.792589,123.046759,127.231708,130.438023,129.339352


In [ ]:
y_test

,Y_01,Y_02,Y_03,Y_04,Y_05,Y_06,Y_07,Y_08,Y_09,Y_10,Y_11,Y_12,Y_13,Y_14
17963,1.326,0.824,0.783,13.284,31.265,17.226,3.664,-25.720,-25.642,-22.282,25.073,-25.509,-25.683,-25.470
14237,1.069,0.737,0.660,13.152,31.978,17.011,3.168,-25.963,-25.848,-21.769,24.791,-26.042,-25.954,-25.801
2992,1.278,1.146,0.974,13.604,32.053,16.582,3.153,-26.120,-25.842,-22.228,24.366,-26.071,-25.970,-26.009
18587,1.476,1.037,1.049,15.259,32.896,17.680,2.976,-25.818,-25.606,-21.670,25.364,-25.684,-25.707,-25.563
19112,1.428,1.083,1.038,17.379,32.569,16.785,3.079,-25.617,-25.460,-21.871,24.694,-25.479,-25.464,-25.652
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25593,1.104,0.755,1.010,11.556,29.433,16.927,3.385,-26.444,-26.302,-22.246,24.573,-26.149,-26.233,-26.073
12105,1.007,0.525,0.376,13.660,26.035,16.337,3.344,-26.236,-26.366,-22.417,23.837,-26.465,-26.530,-26.417
32028,1.250,1.271,1.028,7.183,25.049,14.725,4.096,-27.699,-27.829,-23.862,22.229,-27.674,-27.495,-27.586
26645,1.285,0.927,0.997,16.709,33.185,17.185,3.182,-25.948,-26.199,-22.061,24.914,-26.132,-25.881,-25.894


In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/수연/study/radar/sample_submission.csv')

for idx, col in enumerate(submit.columns):
    if col=='ID':
        continue
    submit[col] = pred[:,idx-1]
print('Done.')

Done.


In [ ]:
len(test_df)

39608

In [ ]:
preds.shape

(13071, 14)

In [ ]:
submit.to_csv('/content/drive/MyDrive/수연/study/radar/submission.csv', index=False)

#Lasso Regression

In [ ]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha=0.005,
                                            random_state=random_state))

score = rmse(lasso)
models_scores.append(['Lasso', score])
print(f'Lasso Score = {score}')

Lasso Score = 0.971810201221009


#ElasticNet Regression

In [ ]:
elastic_net = make_pipeline(RobustScaler(), ElasticNet(alpha=.0005,
                                                        l1_ratio= .9,
                                                        random_state=random_state))

score = rmse(elastic_net)
models_scores.append(['ElasticNet', score])
print(f'ElasticNet Score={score}')

ElasticNet Score=0.9706706830198836


#KernelRidge Regression

In [ ]:
# kernel_ridge= KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)
# score = rmse(kernel_ridge)
# models_scores.append(['KernelRidge', score])
# print(f'KernelRidge Score= {score}')

In [ ]:
# Ranking the scores of each model
pd.DataFrame(models_scores).sort_values(by=[1], ascending=False)

,0,1
1,Lasso,0.971810
0,Linear Regression,0.970781
2,ElasticNet,0.970671


In [ ]:
def lg_nrmse(gt, preds):
    # 각 Y Feature별 NRMSE 총합
    # Y_01 ~ Y_08 까지 20% 가중치 부여
    all_nrmse = []
    for idx in range(1,15): # ignore 'ID'
        rmse = metrics.mean_squared_error(gt[:,idx], preds[:,idx], squared=False)
        nrmse = rmse/np.mean(np.abs(gt[:,idx]))
        all_nrmse.append(nrmse)
    score = 1.2 * np.sum(all_nrmse[:8]) + 1.0 * np.sum(all_nrmse[8:14])
    return score

#Feature Selection

상관관계가 있는 feature 선택, 일변량 feuatre 선택, 재귀 feuature 제거, 교차 검증을 통한 재귀 feature제거 및 트리 기반 선택 방법과 random forest 와 함께 사용된다. 이외에도 PCA와 같이 성분의 수를 관찰하는데에 사용된다.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # data visualization library  
import matplotlib.pyplot as plt
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import time
from subprocess import check_output

import warnings
# ignore all warnings
warnings.filterwarnings('ignore')
# Any results you write to the current directory are saved as output.

data에서 중요한 점 

1) 분류에 사용할 수 없는 id 가 있음. 


In [ ]:
len(X.columns)

56

In [ ]:
data_dia = y
data = X

#standardization
data_n_2 = (data-data.mean())/(data.std())
data=pd.concat([y, data_n_2.iloc[:, 0:10]], axis=1)
data=pd.melt(data, var_name = 'features', value_name='y')

plt.figure(figsize=(56,56))
sns.violinplot(X='features', y= 'value', data=data,split=True, inner='quart')
plt.xticks(rotation=90)

ValueError: ignored

<Figure size 4032x4032 with 0 Axes>

In [ ]:
X_train

,X_01,X_02,X_03,X_04,X_05,X_06,X_07,X_08,X_09,X_10,...,X_47,X_48,X_49,X_50,X_51,X_52,X_53,X_54,X_55,X_56
0,70.544,103.320,67.47,1,101.892,74.983,29.45,62.38,245.71,0.0,...,1,1,9706.03,137.043591,135.359219,147.837968,134.313475,125.605427,136.721425,125.028256
1,69.524,103.321,65.17,1,101.944,72.943,28.73,61.23,233.61,0.0,...,1,1,10423.43,133.736691,135.979817,149.924692,123.630583,127.893337,143.322659,124.877308
2,72.583,103.320,64.07,1,103.153,72.943,28.81,105.77,272.20,0.0,...,1,1,10948.53,132.805112,131.055355,146.814592,128.939070,127.012195,140.395688,122.238232
3,71.563,103.320,67.57,1,101.971,77.022,28.92,115.21,255.36,0.0,...,1,1,15007.03,134.138760,133.239422,139.720132,132.260824,130.723186,147.624829,134.875225
4,69.524,103.320,63.57,1,101.981,70.904,29.68,103.38,241.46,0.0,...,1,1,11051.03,142.728970,136.620022,134.853555,134.760252,125.647793,139.331105,123.272762
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39602,66.465,103.320,62.27,1,103.150,66.825,30.20,77.83,298.05,0.0,...,1,1,60630.73,129.965741,130.807148,133.481737,125.273130,121.780933,133.780110,129.029812
39603,66.465,103.321,62.77,1,102.021,66.825,29.21,102.25,270.67,0.0,...,1,1,60763.43,127.633885,120.158764,142.667802,122.465490,122.987209,143.090741,122.811413
39604,68.504,103.320,64.67,1,103.144,68.864,29.96,102.61,198.07,0.0,...,1,1,8813.33,132.501286,136.893025,134.419328,129.115431,130.920147,140.489232,119.166699
39605,66.465,103.320,63.67,1,102.025,67.845,30.30,112.60,275.52,0.0,...,1,1,62222.33,128.189679,121.495930,141.288011,130.141676,125.518825,136.603634,124.525929


In [ ]:
from sklearn.preprocessing import PolynomialFeatures
X_new_poly = poly_features.transform(X_train)

NameError: ignored

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

def LR(poly_X, X_train, y_train):
  SLR_model = LinearRegression()
  model = LinearRegression()

  SLR_model.fit(X_train,y_train)
  model.fit(poly_X,y_train)

  print("w1: ", model.coef_[0][0])
  print("w2: ", model.coef_[0][1])
  print("b: ", model.intercept_[0])

  SLR_result =SLR_model.predict(X_train)
  result=model.predict(poly_X)

  # plt.figure(figsize=(56,14))
  # plt.scatter(X_train,y_train)
  # plt.plot(X, result, color='red', label='Polynomial Regression')
  # plt.plot(X, SLR_result, color='green', label="Simple Linear Regression")
  # plt.suptitle("LR function", size=60)
  # plt.legend()
  # plt.show()

  return result


X_train, X_poly=poly(X)
result = LR(X_poly, X_train, y_train)
data = np.concatenate((X_train, y_train, result), axis=1)
df = pd.DataFrame(data, columns=['X', 'y', 'predict'])


print("결정계수: ", r2_score(y, result))
print("상관계수 : \n", df.corr())
print("MSE: ",mean_squared_error(y,result))

ValueError: ignored

#Ensemble Modeling

In [ ]:
def begging_predictions(estimator):
  regr = BaggingRegressor(base_estimator=estimator,
                          n_estimators=10,
                          max_samples =1.0,
                          bootstrap=True,
                          n_jobs=n_jobs,
                          random_state=random_state).fit(X_train,y_train)
  br_y_pred=regr.predict(X_test)

  rmse_val = mean_squared_error(y_test, br_y_pred, squared=False) #squared=False > returns Root Mean square Error

  print(f'RMSE for base estimator {regr.base_estimator_}={rmse_val}\n')
  return br_y_pred

In [ ]:
predictions = np.column_stack((begging_predictions(linear_regression),
                               begging_predictions(lasso),
                               begging_predictions(elastic_net)
                               #begging_predictions(kernel_ridge)
                               ))
print(f'bagged predictions shape:{predictions.shape}')

RMSE for base estimator Pipeline(steps=[('linearregression', LinearRegression())])=0.9708333495215723

RMSE for base estimator Pipeline(steps=[('robustscaler', RobustScaler()),
                ('lasso', Lasso(alpha=0.005, random_state=42))])=0.971632328094954

RMSE for base estimator Pipeline(steps=[('robustscaler', RobustScaler()),
                ('elasticnet',
                 ElasticNet(alpha=0.0005, l1_ratio=0.9, random_state=42))])=0.9707081052213155

bagged predictions shape:(13071, 42)
